# 라이브러리 불러오기
- 필요한 라이브러리 가져오기

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import random

#from ggplot import *
#from scipy.stats import logistic
#from sklearn.metrics import roc_curve
#from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix, mean_squared_error
#from sklearn.model_selection import train_test_split
#from sklearn import preprocessing
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import *
#from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
#import graphviz
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.svm import SVC, SVR

# 한글 폰트 실행
from matplotlib import rcParams
from matplotlib import font_manager,rc
rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

# 분석에 도움주는 주요 기본 함수 정의

## 메모리 줄여주는 함수

In [2]:
import pandas as pd 
import numpy as np

def reduce_mem_usage(df, float16_as32=True):
    # memory_usage()는 데이터프레임의 각 열의 메모리 사용량을 계산하고, sum은 그 값을 합산합니다. B -> KB -> MB로 변환합니다.
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터프레임의 초기 메모리 사용량은 {:.2f} MB입니다.'.format(start_mem))

    for col in df.columns:  # 각 열의 이름을 반복합니다.
        col_type = df[col].dtype  # 열의 데이터 타입을 가져옵니다.
        if col_type != object and str(col_type) != 'category':  # 객체 타입이 아닌 수치형 변수만 처리합니다.
            c_min, c_max = df[col].min(), df[col].max()  # 해당 열의 최소값과 최대값을 계산합니다.
            if str(col_type)[:3] == 'int':  # 정수형 변수일 경우 (int8, int16, int32, int64 모두 포함)
                # 이 열의 값 범위가 int8의 범위에 있을 경우 타입을 int8로 변환합니다. (-128부터 127까지)
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                # 이 열의 값 범위가 int16의 범위에 있을 경우 타입을 int16으로 변환합니다. (-32,768부터 32,767까지)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                # 이 열의 값 범위가 int32의 범위에 있을 경우 타입을 int32로 변환합니다. (-2,147,483,648부터 2,147,483,647까지)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                # 이 열의 값 범위가 int64의 범위에 있을 경우 타입을 int64로 변환합니다. (-9,223,372,036,854,775,808부터 9,223,372,036,854,775,807까지)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:  # 실수형일 경우
                # 값이 float16의 범위에 있을 경우 타입을 변환합니다. 더 높은 정밀도가 필요하면 float32를 고려합니다.
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:  # 더 높은 정밀도가 필요하면 float32를 선택할 수 있습니다.
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)
                # 값이 float32의 범위에 있을 경우 타입을 float32로 변환합니다.
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                # 값이 float64의 범위에 있을 경우 타입을 float64로 변환합니다.
                else:
                    df[col] = df[col].astype(np.float64)
    # 최종 메모리 사용량을 계산합니다.
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 데이터프레임의 메모리 사용량은: {:.2f} MB입니다.'.format(end_mem))
    # 초기 메모리 사용량과 비교하여 몇 퍼센트 감소했는지 계산합니다.
    print('메모리 사용량이 {:.1f}% 감소했습니다.'.format(100 * (start_mem - end_mem) / start_mem))

    return df

## 시간 체크 함수

In [4]:
import time

def execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Execution time for {func.__name__}: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
        return result
    return wrapper

## 데이터 가공 함수

In [16]:
@execution_time
def clean_df(data):
    data['고객ID'] = data['고객ID'].astype(str)
    data = reduce_mem_usage(data)
    return data

In [3]:
DATA_PATH = './dataset/CARD/'
DATA_PATH_2 = './dataset/CUSTOMER/'
card_21 = pd.read_csv(DATA_PATH + 'card_21.csv', encoding="euc-kr")
#card_22 = pd.read_csv(DATA_PATH + 'card_22.csv', encoding="euc-kr")
#card_23 = pd.read_csv(DATA_PATH + 'card_23.csv', encoding="euc-kr")
customer_21 = pd.read_csv(DATA_PATH_2 + 'customer_21.csv', encoding="euc-kr")
#customer_22 = pd.read_csv(DATA_PATH_2 + 'customer_22.csv', encoding="euc-kr")
#customer_23 = pd.read_csv(DATA_PATH_2 + 'customer_23.csv', encoding="euc-kr")

In [13]:
temp = card_21.sample(n = 100).reset_index(drop=True)
temp.head()

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,202110,8851293045,서양음식,대구광역시,중구,5번이하,10000
1,202104,9044834464,인터넷P/G,서울특별시,중구,5번이하,20000
2,202107,7184189897,기타음료식품,대구광역시,동구,5번이하,10000
3,202111,6269770133,제과점,경상북도,경산시,5번이하,10000
4,202110,4322838100,스넥,대구광역시,수성구,5번이하,10000


In [17]:
temp2 = clean_df(card_21)
temp2.info()

데이터프레임의 초기 메모리 사용량은 2777.37 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 2777.37 MB입니다.
메모리 사용량이 0.0% 감소했습니다.
Execution time for clean_df: 0h 0m 2.97s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60672642 entries, 0 to 60672641
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   거래년월      int32 
 1   고객ID      object
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 2.7+ GB


In [20]:
temp3 = clean_df(customer_21)
temp3.head(1)

데이터프레임의 초기 메모리 사용량은 2066.56 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 1510.18 MB입니다.
메모리 사용량이 26.9% 감소했습니다.
Execution time for clean_df: 0h 0m 7.98s


,기준년월,고객ID,연령대,성별,고객등급,자택_시도,자택_시군구,수신_요구불예금,수신_거치식예금,수신_적립식예금,수신_펀드,수신_외화예금,대출금액
0,202112,9999998161,60대이상,여,일반등급,대구광역시,달서구,1400000,0,0,0,0,0


In [21]:
customer_21.head(1)

,기준년월,고객ID,연령대,성별,고객등급,자택_시도,자택_시군구,수신_요구불예금,수신_거치식예금,수신_적립식예금,수신_펀드,수신_외화예금,대출금액
0,202112,9999998161,60대이상,여,일반등급,대구광역시,달서구,1400000,0,0,0,0,0


In [22]:
customer_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20835992 entries, 0 to 20835991
Data columns (total 13 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   기준년월      int32 
 1   고객ID      object
 2   연령대       object
 3   성별        object
 4   고객등급      object
 5   자택_시도     object
 6   자택_시군구    object
 7   수신_요구불예금  int32 
 8   수신_거치식예금  int32 
 9   수신_적립식예금  int32 
 10  수신_펀드     int32 
 11  수신_외화예금   int32 
 12  대출금액      int32 
dtypes: int32(7), object(6)
memory usage: 1.5+ GB


In [23]:
card_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60672642 entries, 0 to 60672641
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   거래년월      int32 
 1   고객ID      object
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 2.7+ GB


In [24]:
card_22 = pd.read_csv(DATA_PATH + 'card_22.csv', encoding="euc-kr")
temp4 = clean_df(card_22)
temp4.info()

데이터프레임의 초기 메모리 사용량은 3361.67 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 2881.43 MB입니다.
메모리 사용량이 14.3% 감소했습니다.
Execution time for clean_df: 0h 0m 17.45s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62945850 entries, 0 to 62945849
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   거래년월      int32 
 1   고객ID      object
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 2.8+ GB


In [25]:
# 고객 - 고객id, 연령대만 남기고 삭제
customer_21_test2 = customer_21.loc[:, ['고객ID', '연령대']]
customer_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20835992 entries, 0 to 20835991
Data columns (total 13 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   기준년월      int32 
 1   고객ID      object
 2   연령대       object
 3   성별        object
 4   고객등급      object
 5   자택_시도     object
 6   자택_시군구    object
 7   수신_요구불예금  int32 
 8   수신_거치식예금  int32 
 9   수신_적립식예금  int32 
 10  수신_펀드     int32 
 11  수신_외화예금   int32 
 12  대출금액      int32 
dtypes: int32(7), object(6)
memory usage: 1.5+ GB


In [ ]:
# 고객 데이터 - 60대
customer_21_test2_60 = customer_21.loc[:, ['고객ID', '연령대']]
customer_21_test2_60.head(3)

customer_test_21_60 = customer_test_21[customer_test_21['연령대'].isin(['60대이상'])]

In [ ]:
# 고객 데이터 - 50대
customer_21_test2_50 = customer_21.loc[:, ['고객ID', '연령대']]
customer_21_test2_60.head(3)

customer_test_21 = customer_21.drop(columns=['기준년월', '성별','고객등급', '자택_시도', '자택_시군구', '수신_요구불예금','수신_거치식예금','수신_적립식예금','수신_펀드', '수신_외화예금','대출금액'])

customer_test_21_60 = customer_test_21[customer_test_21['연령대'].isin(['60대이상'])]

card_test_21_id = card_21.drop(columns=['가맹점_시군구', '거래년월', '가맹점_광역시도','승인건수','가맹점업종명','승인금액'])

card_customer_21_60_id= pd.merge(customer_test_21_60, card_test_21_id, on='고객ID', how='inner')
card_customer_21_60_id.head(3)

card_21_test2 = card_21.loc[:, ['고객ID', '가맹점업종명','승인금액']]
card_21_test2.head(3)

In [26]:
card_21.head()

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,202112,4080934272,일반한식,경상북도,경주시,5번이하,20000
1,202112,4116127380,일반한식,경상북도,경주시,5번이하,20000
2,202112,2914424833,일반한식,경상북도,경주시,5번이하,20000
3,202112,2950007638,일반한식,경상북도,경주시,5번이하,60000
4,202112,2931907302,일반한식,경상북도,경주시,5번이하,10000


### 승인건수 정체 알아보기

In [27]:
temp_card_21 = card_21.loc[:, ['고객ID', '가맹점업종명','가맹점_광역시도','가맹점_시군구','승인건수']]
temp_card_21.head(2)

,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수
0,4080934272,일반한식,경상북도,경주시,5번이하
1,4116127380,일반한식,경상북도,경주시,5번이하


In [29]:
temp_card_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60672642 entries, 0 to 60672641
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      object
 1   가맹점업종명    object
 2   가맹점_광역시도  object
 3   가맹점_시군구   object
 4   승인건수      object
dtypes: object(5)
memory usage: 2.3+ GB


In [28]:
temp_card_21_sort = temp_card_21.sort_values(by=['고객ID'], ascending=[True])
temp_card_21_sort.head(10)

,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수
1688392,2800789394,서양음식,부산광역시,부산진구,5번이하
2933054,2800789394,일반한식,부산광역시,부산진구,5번이하
4121409,2801555204,제과점,부산광역시,해운대구,5번이하
4125788,2801555204,일반서적,대구광역시,수성구,5번이하
5178279,2801835265,슈퍼마켓,경상북도,영천시,5번이하
566821,2801881648,일반가구,경상북도,포항시 북구,5번이하
2234778,2801881648,슈퍼마켓,경상북도,포항시 북구,5번이하
116954,2802511760,가례서비스업,경상북도,구미시,5번이하
2372730,2802511760,종합병원,대구광역시,동구,5번이하
3598003,2802511760,인터넷P/G,서울특별시,구로구,5번이하


In [35]:
temp_card_21_sort = temp_card_21.sort_values(by=['고객ID'], ascending=[False])
temp_card_21_sort.head(10)

,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수
6429999,9999998161,택시,대구광역시,수성구,5번초과 10번이하
31594109,9999998161,일반한식,대구광역시,서구,5번이하
6314344,9999998161,슈퍼마켓,대구광역시,달서구,5번이하
54841525,9999998161,제과점,대구광역시,서구,5번이하
11651656,9999998161,택시,대구광역시,수성구,10번초과 15번이하
30040187,9999998161,택시,대구광역시,수성구,5번이하
57268598,9999998161,농·축·수산품,대구광역시,서구,5번이하
60484885,9999998161,기타음료식품,대구광역시,서구,5번이하
28294344,9999998161,슈퍼마켓,대구광역시,서구,5번이하
59537590,9999998161,치과의원,대구광역시,서구,5번이하


In [31]:
customer_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20835992 entries, 0 to 20835991
Data columns (total 13 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   기준년월      int32 
 1   고객ID      object
 2   연령대       object
 3   성별        object
 4   고객등급      object
 5   자택_시도     object
 6   자택_시군구    object
 7   수신_요구불예금  int32 
 8   수신_거치식예금  int32 
 9   수신_적립식예금  int32 
 10  수신_펀드     int32 
 11  수신_외화예금   int32 
 12  대출금액      int32 
dtypes: int32(7), object(6)
memory usage: 1.5+ GB


In [32]:
customer_21['수신_거치식예금'].value_counts()

수신_거치식예금
0           19355039
10000000      148046
20000000       91533
50000000       83356
30000000       71794
              ...   
7900000          911
97000000         908
9800000          901
9900000          898
99000000         698
Name: count, Length: 190, dtype: int64

In [34]:
# 시각화 -> 의미있게 하려면 0을 지우고 보자

In [33]:
# 0을 지우자....
customer_21['수신_거치식예금']

0                  0
1           10000000
2           30000000
3                  0
4                  0
              ...   
20835987           0
20835988           0
20835989           0
20835990           0
20835991           0
Name: 수신_거치식예금, Length: 20835992, dtype: int32